# Analyse des données slavevoyages

URL de la requête : https://www.slavevoyages.org/voyage/database
        
Filtres : 
- filtre "Place where voyage began" : La Rochelle, Rochefort, Bordeaux, Nantes
- Export en csv

J'ai ensuite exporté un csv avec les colonnes suivantes : Voyage itinerary imputed port where began (ptdepimp) place,"Voyage dates that began (depb,a,c)"

Je l'ai simplifié au niveau des colonnes.

Puis chargement du fichier :

In [ ]:
import csv
import re


voyages = []
with open('slavevoyages_exports_filtered.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        year = re.search("([\d]{4})", row["date_departure"] if "date_departure" in row else '')
        year = year.group(0) if year else None
        row["port"] = "La Rochelle et Rochefort" if (row["port"] == 'La Rochelle' or row['port'] == 'Rochefort') else row['port']
        voyages.append({**row, "year": year, "has_year": "année renseignée" if year is not None else "pas d'année"})

In [ ]:
from vega import VegaLite
import pandas as pds

VegaLite({
    "title": "Complétude",
    "mark": "bar",
    "encoding": {
        "column": {
            "type": "nominal", 
            "field": "port",
            "title": "port"
        },
        "y": {
            "field": "has_year",
            "type": "nominal",
            "title": "année de départ renseignée"
        },
        "x": {
            "field": "has_year",
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de voyages"
        },
    }
}, pds.DataFrame(voyages))

In [ ]:
VegaLite({
    "title": "Évolution des départs de navires négriers",
    "mark": "line",
    "width": 800,
    "encoding": {
        "row": {
            "type": "nominal", 
            "field": "port",
            "title": "port"
        },
        "y": {
            "field": "year",
            "type": "quantitative",
            "aggregate": "count",
            "title": "nombre de voyages"
        },
        "x": {
            "field": "year",
            "type": "temporal"
        },
    }
}, pds.DataFrame([v for v in voyages if v["year"] is not None]))